In [1]:
# from ipywidgets import *
# from IPython.core.display import display



from ipywidgets import *
from IPython.display import IFrame, YouTubeVideo
from IPython.core.display import display
from ipywidgets import HTML
import subprocess, sys
import base64

import cv2
import io
import numpy as np
import PIL.Image
from PIL import *

In [2]:
gamma = 1
cl = 0
fo = 0

In [3]:
                #################################
def compress_to_bytes(data, fmt):
    """
    Helper function to compress image data via PIL/Pillow.
    """
    buff = io.BytesIO()
    img = PIL.Image.fromarray(data)    
    img.save(buff, format=fmt)
    return buff.getvalue()
##########

In [4]:
#############################################
def adjust_brightness(img, brightness_factor):
    """Adjust brightness of an Image.
    Args:
        img (numpy ndarray): numpy ndarray to be adjusted.
        brightness_factor (float):  How much to adjust the brightness. Can be
            any non negative number. 0 gives a black image, 1 gives the
            original image while 2 increases the brightness by a factor of 2.
    Returns:
        numpy ndarray: Brightness adjusted image.
    """
    table = np.array([ i*brightness_factor for i in range (0,256)]).clip(0,255).astype('uint8')
    if img.shape[2]==1:
        return cv2.LUT(img, table)[:,:,np.newaxis]
    else:
        return cv2.LUT(img, table) 
##################################################

In [5]:
def Travail_img(gamma):
    im_in = PIL.Image.open(io.BytesIO(Img.value))
    im_array = np.array(im_in)#[..., :3]
#     imm = np.zeros(im_array.shape[:2],np.uint8)
    imm = cv2.cvtColor(im_array, cv2.COLOR_BGRA2BGR)
    imm=adjust_brightness(imm, gamma)
    #########################################"
    fmt = 'png'
    data_comp = compress_to_bytes(imm, fmt)
    Img.value=data_comp
    return gamma
    #####

In [6]:
#########################               
def Contrast(_):
    global gamma
    cl=clair_z.value
    fo=sombre_z.value
    cl_new=cl_new_z.value
    fo_new=fo_new_z.value
    if cl>cl_new:
        if fo==0:
            cl_new+=1
            cl_new_z.value=cl_new #
        else:
            clair_z.value=cl_new
            fo_new=fo_new-1
            fo_new_z.value=fo_new
            sombre_z.value=fo_new

    if fo>fo_new:
        if cl==0:
            fo_new+=1
            fo_new_z.value=fo_new
        else:
            sombre_z.value=fo_new
            cl_new=cl_new-1
            cl_new_z.value=cl_new
            clair_z.value=cl_new

    if fo==0:
        gamma=float(cl/10)+1
    if cl==0:
        gamma=1-float(fo/10)
    Travail_img(gamma)
#########################

In [7]:
print("coucou ! Hello")

coucou !


In [8]:
file = open("Lib2.png", "rb")
image = file.read()
Img = widgets.Image(
    value=image,
    format='png',
    width=300,
    height=400,
)

In [9]:
clair_z = widgets.IntSlider(min = 0, max = 60,  value = 0 , description='+ clair')
sombre_z = widgets.IntSlider(min = 0, max = 10,  value = 0 , description='+ sombre')
cl_new_z = widgets.IntSlider(min = 0, max = 10,  value = 0 , description='cl_new')
fo_new_z = widgets.IntSlider(min = 0, max = 10,  value = 0 , description='fo_new')
clair_z.observe(Contrast, names='value')
sombre_z.observe(Contrast, names='value')

In [10]:
Menus = HBox([Img, clair_z,sombre_z], layout=Layout(max_width='100%',display='inline-flex',flex_flow='row wrap'))
display(Menus)

NameError: name 'clair_z' is not defined

In [ ]:
# Menus = HBox([Img], layout=Layout(max_width='100%',display='inline-flex',flex_flow='row wrap'))
# display(Menus)

In [ ]:
print("youpi")